### Using ticdat to convert data between different schemas
Here I'm showing how I convert  the testing data when the `input_schema` changes between versions of the package under development (i.e. `tts_netflow_b`).

This notebook needs `tts_netflow_b` in the Python path, and `test_tts_netflow_b` as a subdirectory of the current directory. If you run it directly from the `notebooks` directory cloned from GitHub it won't find `test_tts_netflow_b`. I prefer to use symbolic links for such things.

In [1]:
import tts_netflow_b
import os
from ticdat import TicDatFactory

Since we are demonstrating tts_netflow_b as a dynamic work in progress, lets record the version.

In [2]:
tts_netflow_b.__version__

'0.0.2'

Only one file to convert here, but I'll demonstrate in a way that extends to more than one.

In [3]:
files =  os.listdir(os.path.join("test_tts_netflow_b", "data"))
files

['sample_data.json']

This notebook is being run when `tts_netflow_b.input_schema` reflects the new schema (i.e. the 0.0.2 schema), but the data in `files` represents the old schema (i.e. the 0.0.1 schema). 

To convert, I will thus need the old schema. This is easy, since `tts_netflow_b.modeling_schema` is in fact the old schema.

In [4]:
for f in files:
    dat = tts_netflow_b.modeling_schema.json.create_pan_dat(
        os.path.join("test_tts_netflow_b", "data", f))
    # I recommend two monitors so as to study both schemas at the same time
    new_dat = tts_netflow_b.input_schema.PanDat(nodes=dat.nodes, commodities=dat.commodities, 
                                                arcs=dat.arcs, cost=dat.cost, 
                                                supply=dat.inflow[dat.inflow["Quantity"] > 0],
                                                demand=dat.inflow[dat.inflow["Quantity"] < 0])
    new_dat.demand["Quantity"] = -new_dat.demand["Quantity"]
    # make sure its kosher before you write it back
    assert not tts_netflow_b.input_schema.find_duplicates(new_dat) 
    assert not tts_netflow_b.input_schema.find_foreign_key_failures(new_dat)
    assert not tts_netflow_b.input_schema.find_data_type_failures(new_dat)
    assert not tts_netflow_b.input_schema.find_data_row_failures(new_dat)
    tts_netflow_b.input_schema.json.write_file(new_dat, 
                                               os.path.join("test_tts_netflow_b", "data", f))    